In [1]:
import os
import sys
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform
import pathlib
import numpy as np
import random
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Windows-10-10.0.22621-SP0
Tensor Flow Version: 2.10.0
Keras Version: 2.10.0

Python 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
Pandas 1.5.1
Scikit-Learn 1.1.3
SciPy 1.9.3
GPU is available


In [2]:
# Variables and params
batch_size = 128
img_height = 100
img_width = 100

train_root = pathlib.Path('../FruitScale/data/Training')
train_root = train_root.resolve()

test_root = pathlib.Path('../FruitScale/data/Test')
test_root = test_root.resolve()

fireTest_root = pathlib.Path('../FruitScale/data/FireTest')
fireTest_root = fireTest_root.resolve()

image_count = len(list(train_root.glob('*/*.jpg')))

random.seed(42)
#print(train_root)

In [3]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_root,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 67692 files belonging to 131 classes.
Using 54154 files for training.


In [4]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_root,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
print(val_data)

Found 67692 files belonging to 131 classes.
Using 13538 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [5]:
class_names = train_data.class_names
num_classes = len(class_names)
print(num_classes)

131


In [6]:
## TODO: rotate, change brighness, rescale, zoom
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    validation_split=0.2,
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5,1.1],
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,

)

train_generator = datagen.flow_from_directory(
    train_root,
    target_size=(224,224),
    subset="training",
    batch_size = 200,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    train_root,
    target_size=(224,224),
    subset="validation",
    batch_size = 200,
    class_mode='categorical'
)

testDatagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
)

test_generator = testDatagen.flow_from_directory(
    test_root,
    target_size=(224,224),
    #batch_size = 128,
    #class_mode='categorical'
)

Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [7]:
imgs,labels=next(train_generator)
np.shape(imgs)

(200, 224, 224, 3)

In [8]:
imgs,labels=next(test_generator)
np.shape(imgs)

(32, 224, 224, 3)

In [9]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [10]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=16, kernel_size=(5, 5), input_shape=(224,224,3), activation='relu'))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(131, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 16)      1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 32)      12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 49, 49, 64)        51264     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 64)       0

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=25,
    validation_data = val_generator, verbose=1,
    callbacks=[early_stop],
    batch_size = 200 # BATCH_SIZE
)

Epoch 1/25
 38/271 [===>..........................] - ETA: 8:24 - loss: 4.3928 - accuracy: 0.0379

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(test_generator)
print('Loss:', loss)
print('Accuracy:', acc)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

Y_pred = model.predict_generator(test_generator, 22687)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

#predictions = model.predict(test_generator)
#print(classification_report(test_generator.classes,np.round(predictions)))

In [ ]:
print(confusion_matrix(test_generator.classes.data, np.round(predictions)))

In [ ]:
import cv2

x=random.choice(os.listdir(str(test_root)+'/Banana'))
print(x)
img=plt.imread(os.path.join(str(test_root)+'/Banana',x))
img=cv2.resize(img,dsize=(224,224))
img_new=keras.applications.mobilenet.preprocess_input(img)
plt.imshow(img)

In [ ]:
model.predict(np.expand_dims(img_new,0)).round(2)

In [ ]:
index = train_generator.class_indices
print (index)

In [ ]:
x=random.choice(os.listdir(str(fireTest_root)))
print(x)
img=plt.imread(os.path.join(str(fireTest_root),x))
img=cv2.resize(img,dsize=(224,224))
img_new=keras.applications.mobilenet.preprocess_input(img)
plt.imshow(img)

In [ ]:
predictions = model.predict(np.expand_dims(img_new,0)).round(2)[0]
i = 0
for prediction in predictions:
    if prediction > 0.0:
        k = 0
        for a,b in index.items():
            if k == i:
                print(a + ": " + str(prediction))
            k = k + 1
    i = i+1

In [ ]:
model.save("trained_mobilenet.h5")

In [ ]:
new_model = tf.keras.models.load_model('trained_mobilenet.h5')

In [ ]:
new_model.summary()